In [1]:
import os
import time
import tqdm

import numpy as np
import pandas as pd
import audb
import audiofile
from scipy.io.wavfile import read as read_wav
from matplotlib import pyplot as plt
import opensmile

import sklearn 
from sklearn.model_selection import train_test_split #train/test split 
from sklearn.svm import SVC #Model
from sklearn.metrics import confusion_matrix #Metrics 
from sklearn.metrics import f1_score 
from sklearn.model_selection import cross_val_score #Cross-validation
from sklearn.model_selection import KFold

import scipy
import random #for shuffling values 
from scipy import stats

#List of prosodic features we will observe

features_prosody= ['Loudness_sma3', 
                   'hammarbergIndex_sma3',
                   'F0semitoneFrom27.5Hz_sma3nz',
                    'slope0-500_sma3',
                  'slope500-1500_sma3',
                   'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'mfcc1_sma3',
 'mfcc2_sma3',
 'mfcc3_sma3',
 'mfcc4_sma3']

# Function to perform one-hot encoding
def encode(label):
    if 'non-hate' in label:
        return 0
    elif 'hate' in label:
        return 1

#Gets low-level features of all audios 
def lowlevel_features(list_with_files):
    path= "data/clean_audio1/"
    
    #  Define feature extractor
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    )

    df_data= []

    for i in (list_with_files):
        f= path + i
        sampling_rate, data = read_wav(f) #gets sampling rate and dimensions of data
        #print(f"Sampling rate of {f}:{sampling_rate}")
        
        db = audb.load('emodb',
        version='1.1.1',
        format='wav',
        mixdown=True,
        sampling_rate=sampling_rate,
        media='wav/03a01.*',  # load subset
        full_path=False,
        verbose=False,)

        #extract features
        signal, sampling_rate = audiofile.read(f, duration=120, always_2d=True)
        data= smile.process_signal(signal, sampling_rate)
        df_data.append(data.loc[:,features_prosody].values)

   # df= pd.concat(df_data, ignore_index=True)
    return df_data
        
def t_test(dataframe):
    for i in features_prosody:
        class_0= dataframe[i][dataframe['label']==0].values
        class_1= dataframe[i][dataframe['label']== 1].values 
        t_statistic, p_value = stats.ttest_ind(class_0, class_1)
        print(f"T-statistic {i}:", t_statistic)
        print(f"P-value {i}:", p_value)
    

In [2]:
df=pd.read_csv("data/datasets/data_spanish_dataset.csv")

# Encoding categorical variables. Not hate speech = 0. Hate speech = 1. 
df['label'] = df['label'].apply(lambda x: encode(x))

#Storing labels in array 
labels= df['label'].values
print(labels)

[0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1
 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1]


In [3]:
#process wav files 
path="data/clean_audio1"
dir_list = os.listdir(path)
feature_data= lowlevel_features(dir_list)

In [4]:
# Find the shape of the largest matrix
max_shape = max(matrix.shape for matrix in feature_data)

# Zero-pad the rows of matrices
padded_matrices = []
for matrix in feature_data:
    pad_width = ((0, max_shape[0] - matrix.shape[0]), (0, 0))
    padded_matrix = np.pad(matrix, pad_width, mode='constant', constant_values=0)
    padded_matrices.append(padded_matrix)


In [5]:
# Check the shape of padded matrices
for matrix in padded_matrices:
    print(matrix.shape)

(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(11996, 12)
(119

In [35]:
#Train/Test split 
y= labels
#y= y_axis.to_numpy().flatten() #Labels (class 0 and 1)
X = np.asarray(padded_matrices)
X= X.reshape(-1, X.shape[1]*X.shape[2])
#X= X_axis.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [33]:
print()
print(y_train.shape)

(64, 143952)
(64,)


In [41]:
#Cross-validation recall

C_list= np.logspace(-5, 5, 20)
gamma_list= np.logspace(-5, 5, 20)
recall = np.zeros((len(C_list), len(gamma_list)))


for i in range(len(C_list)):
     print(f'Iteration {i}')
     for j in range(len(gamma_list)):
        svc= SVC(C=C_list[i], gamma=gamma_list[j], kernel='rbf')
        svc.fit(X_train, y_train)
        y_pred= svc.predict(X_val)
        tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
        recall[i, j] = float(tp+tn) / (tp + fn + fp + tn)
               

index_max = np.unravel_index(np.argmax(recall), recall.shape)

print('Max. recall:{}'.format(recall[index_max]))
C_opt = C_list[index_max[0]]
gamma_opt = gamma_list[index_max[1]]
print('Optimal C: {}'.format(C_opt))
print('Optimal gamma:{}'.format(gamma_opt))

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Max. recall:0.5
Optimal C: 1e-05
Optimal gamma:1e-05


In [42]:
svc= SVC(kernel='rbf', class_weight='balanced', C=C_opt, gamma=gamma_opt)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[6, 4],
       [5, 5]], dtype=int64)

In [43]:
#Validation of model (recall)

kf = KFold(n_splits=10, shuffle=True)
scores = [] 
for train_idx, val_idx in kf.split(X):
    X_train_k, y_train_k = X[train_idx], y[train_idx]
    X_val_k, y_val_k = X[val_idx], y[val_idx]
    svc= SVC(C=C_opt, gamma=gamma_opt, kernel='rbf', class_weight='balanced')
    svc.fit(X_train_k, y_train_k)
    y_pred_k= svc.predict(X_val_k)
    cm= confusion_matrix(y_val_k, y_pred_k)
    tn, fp, fn, tp = confusion_matrix(y_val_k, y_pred_k).ravel()
    recall_v = float(tp+tn) / (tp + fn + fp + tn)
    scores.append(recall_v)

print('Recall mean: {}'.format(np.mean(scores))) 
print('Recall standard deviation: {}'.format(np.std(scores)))

Recall mean: 0.63
Recall standard deviation: 0.1004987562112089
